<a href="https://colab.research.google.com/github/ar-ushi/YOLO-Object-Detection/blob/master/annotating_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

In [ ]:
os.chdir('/content/drive/My Drive/YOLO_V4/Data/csv_folder')
classes = pd.read_csv('class-descriptions-boxable.csv', header = None)
classes.head()

,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [ ]:
cname = ['Coffee', 'Tea']

#get strings corresponding to cname
strings = []
for i in cname:
  req_class = classes.loc[classes[1] == i]
  string = req_class.iloc[0][0]
  print(string)
  #appending string corresponding to cname into the list
  strings.append(string)

print(strings)

/m/02vqfm
/m/07clx
['/m/02vqfm', '/m/07clx']


In [ ]:
#getting required columns - center,x,y,width and height
annotate = pd.read_csv('train-annotations-bbox.csv', usecols = ['ImageID','LabelName', 'XMin', 'XMax','YMin', 'YMax'])
annotate.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,000002b66c9c498e,/m/01g317,0.012500,0.195312,0.148438,0.587500
1,000002b66c9c498e,/m/01g317,0.025000,0.276563,0.714063,0.948438
2,000002b66c9c498e,/m/01g317,0.151562,0.310937,0.198437,0.590625
3,000002b66c9c498e,/m/01g317,0.256250,0.429688,0.651563,0.925000
4,000002b66c9c498e,/m/01g317,0.257812,0.346875,0.235938,0.385938


In [ ]:
#get records with matching classes
filter_data = annotate.loc[annotate['LabelName'].isin(strings)].copy()
filter_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
4383,0002f921515d14e8,/m/02vqfm,0.050625,0.836250,0.038332,0.630800
4613,00030b70a3abfcea,/m/07clx,0.739375,0.998750,0.000000,0.737488
4615,00030b70a3abfcea,/m/07clx,0.761875,0.997500,0.084042,0.279509
5701,0003d7be45f109cb,/m/02vqfm,0.000000,0.498437,0.239063,0.509375
15674,000c26b44a7b6a22,/m/02vqfm,0.729375,0.761250,0.326742,0.398305


In [ ]:
filter_data['classNumber'] = ''
filter_data['center-x'] = ''
filter_data['center-y'] = ''
filter_data['width'] = ''
filter_data['height'] = ''

In [ ]:
#iterate through all class strings and assign a class number acccording to the order 
for i in range(len(strings)): 
  filter_data.loc[filter_data['LabelName'] == strings[1], 'classNumber'] = i
#calculate center x and center y values
filter_data['center-x'] = (filter_data['XMax'] + filter_data['XMin']) / 2
filter_data['center-y'] = (filter_data['YMax'] + filter_data['YMin']) / 2

#calculate width and height values
filter_data['width'] = filter_data['XMax'] - filter_data['XMin']
filter_data['height'] = filter_data['YMax'] - filter_data['YMin']

yolo_val = filter_data.loc[:,['ImageID','classNumber','center-x', 'center-y','width', 'height']]

In [ ]:
img_path ='/content/drive/My Drive/YOLO_V4/Data/Dataset/train/Coffee_Tea'

os.chdir(img_path)

for current_dir, dirs, files in os.walk('.'):
  for f in files:
    if f.endswith('.jpg'):
      img_title = f[:-4]
      yolo_file = yolo_val.loc[yolo_val['ImageID'] == img_title]

      df = yolo_file.loc[:,['classNumber','center-x', 'center-y','width', 'height']]
      save_path = img_path + '/' + img_title + '.txt'

      df.to_csv(save_path,header=False, index= False,sep=' ')
